# My Wilcoxon Tests

This notebook will cover the basic Wilcoxon data tests.

Author: Alvaro Paricio. sept.2016

## Scenarios


## References
* SCIPY:
    * Lectures: http://www.scipy-lectures.org/packages/statistics/index.html
* MATLAB:
    * Signtest: http://es.mathworks.com/help/stats/signtest.html?searchHighlight=signtest
* Numpy:
    * http://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html
* For Wilcoxon tests:
    * http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.wilcoxon.html
    * https://gist.github.com/mblondel/1761714

http://osdir.com/ml/python-scientific-user/2009-07/msg00014.html

I did a quick comparison between Matlab/stats (R14SP3), R (2.8.1), and Python/SciPy (0.7). Maybe this is somehow useful for others too.

(I’m intentionally violating the continuous distribution assumptions.)

Samples:

A1 <-> B: not paired with ties

A2 <-> B: not paired without ties

A1 <-> C: paired with zeros

A2 <-> C: paired without zeros

- Matlab

      A1 = 0:19

      A2 = A1 + (1:20)./100

      B = 0:39

      C = [0:14,16:20]

 

- R

      A1 <- 0:19

      A2 <- A1 + 1:20/100

      B <- 0:39

      C <- c(0:14,16:20)

 

- SciPy

A1 = numpy.arange(20)

A2 = A1 + numpy.arange(1,21)/100.0

B = numpy.arange(40)

C = numpy.array(range(15) + range(16,21))

 

 

2 Samples, Not Paired

=====================

 

(from scipy.stats import stats)

 

Kruskal-Wallis Test

-------------------

 

Same p-values for all.

 

Samples contain ties:

 

- Matlab: kruskalwallis([A1,B],[A1*0,B*0+1]) = 0.00170615101265

- R: kruskal.test(list(A1,B)) = 0.00170615101265

- R: wilcox.test(A1,B, correct=FALSE) = 0.00170615101265 (+warning: ties)

- SciPy: stats.kruskal(A1,B) = 0.00170615101265

 

(R: kruskal = wilcox without correction for continuity)

 

Samples without ties:

 

- Matlab: kruskalwallis([A2,B], [A2*0,B*0+1]) = 0.00288777919292

- R: kruskal.test(list(A2,B)) = 0.00288777919292

- SciPy: stats.kruskal(A2,B) = 0.00288777919292

     

 

Wilcoxon Rank Sum (aka Mann Whitney U) Test

-------------------------------------------

 

Matlab and R identical (but different defaults wrt exact/approximate),

SciPy computes approximate results and does not correct for continuity (changed in version 7.1 for stats.mannwhitneyu?).

 

Samples contain ties:

 

- Matlab: ranksum(A1,B) = 0.00175235702866

- R: wilcox.test(A1,B) = 0.00175235702866 (+warning: ties)

 

- R: wilcox.test(A1,B,correct=FALSE) = 0.001706151012654 (+warning: ties)

 

- SciPy: stats.mannwhitneyu(A1,B)[1]*2 = 0.0017086895586986284

 

- SciPy: stats.ranksums(A1,B) = 0.0017112312247389294

 

Samples without ties:

 

- Matlab: ranksum(A2,B) = 0.00296255173431

- R: wilcox.test(A2,B, exact=FALSE) = 0.00296255173431

 

- Matlab: ranksum(A2,B,'method','exact') = 0.00246078580826

- R: wilcox.test(A2,B) = 0.00246078580826

 

- R: wilcox.test(A2,B, exact=FALSE, correct=FALSE) = 0.00288777919292

- SciPy: stats.mannwhitneyu(A2,B)[1]*2 = 0.00288777919292

- SciPy: stats.ranksums(A2,B) = 0.00288777919292

 

(SciPy: mannwhitneyu = ranksums = kruskal if no ties)

 

 

2 Samples, Paired, Wilcoxon Sign Rank Test

==========================================
(from scipy.stats import wilcoxon)

Matlab and SciPy do not correct for continuity and R does.

Matlab and R have different defaults for exact/approximate.

Matlab computes exact results also if ties/zeros exist.

With zeros:
- Matlab: signrank(A1,C,'method','approximate') = 0.02534731867747

- R: wilcox.test(A1 - C, correct=FALSE) = 0.02534731867747 (+warnings: ties + zeros)

- Matlab: signrank(A1,C) = 0.06250000000000

- R: wilcox.test(A1 - C) = 0.0368884257070 (+warnings: ties + zeros)

- SciPy: wilcoxon(A1,C) = nan (+error: sample size too small)

Without zeros:

- Matlab: signrank(A2,C,'method','exact') = 0.59581947326660

- R: wilcox.test(A2 - C) = 0.59581947326660


- Matlab: signrank(A2,C) = 0.57548622813650    

- R: wilcox.test(A2 - C, exact=FALSE, correct=FALSE) = 0.57548622813650

- SciPy: wilcoxon(A2,C) = 0.57548622813650


- R: wilcox.test(A2 - C, exact=FALSE) = 0.5882844808893

In [44]:
%matplotlib inline

import numpy as np
import math as mat
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.lines as mlines
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu

import sklearn as sk
import pandas as pd

In [124]:
def check_significance( s, p ):
    return (p > s)

def __signtest1( s, a, b ):
    b2 = [b]*len(a)
    z_stats, p_value = wilcoxon(a,b2)
    h = check_significance(s, p_value)
    return h, p_value, z_stats

def __signtest2( s, a, b ):
    z_stats, p_value = wilcoxon(a,b)
    h = check_significance(s, p_value)
    return h, p_value, z_stats

def __signrank1( s, a, b ):
    b2 = [b]*len(a)
    z_stats, p_value = wilcoxon(a,b2)
    h = check_significance(s, p_value)
    return h, p_value, z_stats

def __signrank2( s, a, b ):
    z_stats, p_value = wilcoxon(a,b)
    h = check_significance(s, p_value)
    return h, p_value, z_stats

def __ranksum1( s, a, b ):
    b2 = [b]*len(a)
    x, y, z_stats, p_value = ranksums(a,b2)
    h = check_significance(s, p_value)
    return h, p_value, z_stats

def __ranksum2( s, a, b ):
    x, y, z_stats, p_value = ranksums(a,b)
    h = check_significance(s, p_value)
    return h, p_value, z_stats

def __ttest1( s, a, b ):
    t_stats, p_value = ttest_1samp(a,b)
    h = check_significance(s, p_value)
    return h, p_value, t_stats

def __ttest2( s, a, b ):
    t_stats, p_value = ttest_rel(a,b)
    h = check_significance(s, p_value)
    return h, p_value, t_stats


# verbose signtest1
# Compare ONE series with test SIGN
# Hipotesis is: MEDIAN of A is B
def __signtest1_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __signtest1( significance, num_a, num_b )
    txt = "SIGN-TEST(median value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" has MEDIAN of "+label_b

    txt +=  "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     z = "+str(z)
    print( txt )
    return h,p,z,txt


# verbose signtest2
# Compare TWO series with test SIGN
# Hipotesis is: MEDIAN of A and B are equal
def __signtest2_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __signtest2( significance, num_a, num_b )
    txt = "SIGN-TEST(median value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" equals MEDIAN of "+label_b

    txt +=  "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     z = "+str(z)
    print( txt )
    return h,p,z,txt


# verbose signrank1
# Compare ONE series with test SIGN
# Hipotesis is: MEDIAN of A is B
def __signrank1_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __signrank1( significance, num_a, num_b )
    txt = "SIGN-RANK(median value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" has MEDIAN of "+label_b

    txt +=  "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     w = "+str(z)
    print( txt )
    return h,p,z,txt


# verbose signrank2
# Compare TWO series with test SIGN
# Hipotesis is: MEDIAN of A and B are equal
def __signrank2_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __signrank2( significance, num_a, num_b )
    txt = "SIGN-RANK(median value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" equals MEDIAN of "+label_b

    txt +=  "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     w = "+str(z)
    print( txt )
    return h,p,z,txt

# verbose ranksum1
# Compare ONE series with RANK SUM TEST
# Hipotesis is: MEAN of A is B
def __ranksum1_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __ranksum1( significance, num_a, num_b )
    txt = "RANK-SUM(median value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" has MEAN of "+label_b

    txt +=  "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     w = "+str(z)
    print( txt )
    return h,p,z,txt


# verbose ranksum2
# Compare TWO series with RANK SUM TEST
# Hipotesis is: MEAN of A and B are equal
def __ranksum2_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __ranksum2( significance, num_a, num_b )
    txt = "RANK-SUM(man value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" equals MEAN of "+label_b

    txt +=  "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     w = "+str(z)
    print( txt )
    return h,p,z,txt

# verbose ttest1
# Compare ONE series with test T-TEST
# Hipotesis is: MEAN VALUE of A is fixed B
def __ttest1_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __ttest1( significance, num_a, num_b )
    txt = "T-TEST(mean value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" has MEAN of "+label_b
    txt += "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     t = "+str(z)
    print( txt )
    return h,p,z,txt

# verbose ttest2
# Compare TWO series with test T-TEST
# Hipotesis is: MEAN VALUE of A and B are equal
def __ttest2_v( hipotesys, label_a, label_b, num_a, num_b ):
    significance = 0.05
    confidence = 1-significance
    h,p,z = __ttest2( significance, num_a, num_b )
    txt = "T-TEST(mean value): "+hipotesys
    if( h ):
        txt += " is TRUE"
    else:
        txt += " is FALSE"
    txt += " --> "+label_a+" equals MEAN of "+label_b
    txt += "\n     confidence = "+str(confidence)+"\n     p = "+str(p)+"\n     t = "+str(z)
    print( txt )
    return h,p,z,txt



In [133]:
def non_parametric_test( hipotesys, label1, label2, test, vals1, vals2 ):
    test_catalog = {'signtest1_v': __signtest1_v,
                    'signtest2_v': __signtest2_v,
                    'signrank1_v': __signrank1_v,
                    'signrank2_v': __signrank2_v,
                    'signrank1'  : __signrank1,
                    'signrank2'  : __signrank2,
                    'ranksum1_v' : __ranksum1_v,
                    'ranksum2_v' : __ranksum2_v,
                    'ranksum1'   : __ranksum1,
                    'ranksum2'   : __ranksum2,
                    'signtest1'  : __signtest1,
                    'signtest2'  : __signtest2,
                    'ttest1_v'   : __ttest1_v,
                    'ttest1'     : __ttest1,
                    'ttest2_v'   : __ttest2_v,
                    'ttest2'     : __ttest2,
                   }
    try:
        return test_catalog[test]( hipotesys, label1, label2, vals1, vals2 )
    except:
        print( "ERROR. test not found: " + test )


# Ejemplos de: "Statistics for Engineers and Scientists"

## Ejemplo Sign-Test UnPaired. Book 16.1
Pagina 656

In [126]:
hedge_trimmer_battery_duration = np.array([1.5, 2.2, 0.9, 1.3, 2.0, 1.6, 1.8, 1.5, 2.0, 1.2, 1.7])

In [127]:
h, p, z, txt = non_parametric_test(
    "HEDGE TRIMMER", "Battery duration", "1.8 hours",
    'signtest1_v', hedge_trimmer_battery_duration, 1.8 )

SIGN-TEST(median value): HEDGE TRIMMER is TRUE --> Battery duration has MEDIAN of 1.8 hours
     confidence = 0.95
     p = 0.138127827574
     z = 13.0


## Ejemplo Sign-Test Paired. Book 16.2
Pagina 659

In [128]:
cars = (
    [4.2, 4.7, 6.6, 7.0, 6.7, 4.5, 5.7, 6.0, 7.4, 4.9, 6.1, 5.2, 5.7, 6.9, 6.8, 4.9],
    [4.1, 4.9, 6.2, 6.9, 6.8, 4.4, 5.7, 5.8, 6.9, 4.9, 6.0, 4.9, 5.3, 6.5, 7.1, 4.8]
)
tires_radial = [4.2, 4.7, 6.6, 7.0, 6.7, 4.5, 5.7, 6.0, 7.4, 4.9, 6.1, 5.2, 5.7, 6.9, 6.8, 4.9]
tires_belted = [4.1, 4.9, 6.2, 6.9, 6.8, 4.4, 5.7, 5.8, 6.9, 4.9, 6.0, 4.9, 5.3, 6.5, 7.1, 4.8]



In [131]:
h, p, z, txt = __signtest2_v( "FUEL CONSUMPTION",
                          "Radial Tires", "Belted Tires", tires_radial, tires_belted)

h, p, z, txt = non_parametric_test(
    "FUEL CONSUMPTION", "Radial Tires", "Belted Tires",
    'signrank2_v', tires_radial, tires_belted )


SIGN-TEST(median value): FUEL CONSUMPTION is FALSE --> Radial Tires equals MEDIAN of Belted Tires
     confidence = 0.95
     p = 0.0376353137873
     z = 19.5
SIGN-RANK(median value): FUEL CONSUMPTION is FALSE --> Radial Tires equals MEDIAN of Belted Tires
     confidence = 0.95
     p = 0.0376353137873
     w = 19.5


## Ejemplo Signed-Rank. Book 16.3
Pagina 661

In [130]:
h, p, z, txt = non_parametric_test(
    "HEDGE TRIMMER", "Battery duration", "1.8 hours",
    'signrank1_v', hedge_trimmer_battery_duration, 1.8 )

SIGN-RANK(median value): HEDGE TRIMMER is TRUE --> Battery duration has MEDIAN of 1.8 hours
     confidence = 0.95
     p = 0.138127827574
     w = 13.0


## Ejemplo Signed-Rank. Book 16.4
Pagina 662

In [ ]:
score_with_problems = [ 531, 621, 663, 579, 451, 660, 591, 719, 534, 575 ]
score_without_probl = [ 509, 540, 688, 502, 424, 683, 568, 748, 530, 524 ]

h, p, z, txt = non_parametric_test(
    "EXAMS scoring improve 50pts", "With examples support", "Improved 50",
    'signrank1_v', score_with_problems - score_without_probl, 50 )

# Ejemplos de "Introductory Statistics with R"
"Introductory Statistics with R" by Peter Dalgaard
http://www.academia.dk/BiologiskAntropologi/Epidemiologi/PDF/Introductory_Statistics_with_R__2nd_ed.pdf

MBlondel: https://gist.github.com/mblondel/1761714
Port to Python of examples in chapter 5.

one sample t-test / null hypothesis: expected value = 7725
Description: daily intake of energy in kJ for 11 women

In [74]:
daily_intake = np.array([5260,5470,5640,6180,6390,6515,
                         6805,7515,7515,8230,8770])
h, p, z, txt = non_parametric_test(
    "DAILY INTAKE", "Energy food intake", "equals 7725",
    'ttest1_v', daily_intake, 7725 )

T-TEST(mean value): DAILY INTAKE is FALSE --> Energy food intake has MEAN of equals 7725
     confidence = 0.95
     p = 0.0181372351761
     t = -2.82075406083


# Ejemplos de MATHWORKS

## Ejemplo Sign-Test
http://es.mathworks.com/help/stats/signtest.html?searchHighlight=signtest

In [ ]:
    8.4521    7.8047
   11.6869   11.4094
    4.2009    5.1133
    9.1664   12.1655
    8.0020   10.0300
    5.3285    6.0153
    6.6300    5.1235
    8.0499    8.6737
   18.0763   19.2164
   14.7665   15.3380
    5.2726    8.4187
   15.7798   16.2093
    8.8583    8.5575
    7.2735    7.4783
    8.8347    7.8894

In [9]:
# np.random.randint(0, 100, (3, 6, 5))
a = np.random.randint( 0, 100, 100)
b = np.random.randint( 0, 50, 100)
print(a.__repr__, "\n", a)
print(b.__repr__, "\n", b)

<method-wrapper '__repr__' of numpy.ndarray object at 0x118384440> 
 [46 81 27 32 74  9 57 97 77 24  4 32  4  3  1 33 58 73 84 84 81 11 30 66 93
 43 34 87 96 36 40 59 57 43 42 13 60 49 46 97 12 17 14 96 34 46 19 18 18 38
 28  1 13  2 16 67 86 76 32 54 62 46 41 69 35 74 46 32 46  2 40 98 13 20 62
 90 58 26 49 58 77 80 61  7  3 55 57 66 75 50 93 95 62 55 70 69 89 41 33 84]
<method-wrapper '__repr__' of numpy.ndarray object at 0x1183846c0> 
 [39 17  0 24 49 16 19 31 26  0 20 37  9 24 19 41 12 44 29 33 45 42  3  9 23
 22 39 46 39  3  8 13 47 27  8 11 24  6 31 40 15 10 36 32 30  8 10 43  1 17
  5 10 26 29 29  7 35  8 12  8 15 37 15  4 25  6  6 46 12 34  5 17 17 11  2
 37 14  2 34 39 40 17 38 27 14  4 24 23 39 34 42 17  9  8 11 40 30  5 48 18]


In [39]:
significance = 0.05
# daily intake of energy in kJ for 11 women
daily_intake = np.array([5260,5470,5640,6180,6390,6515,
                         6805,7515,7515,8230,8770])
#print(daily_intake.__repr__, "\n", daily_intake)

# one sample t-test
# null hypothesis: expected value = 7725
t_statistic, p_value = ttest_1samp(daily_intake, 7725)

# p_value < 0.05 => alternative hypothesis:
# data deviate significantly from the hypothesis that the mean
# is 7725 at the 5% level of significance
print( "one-sample t-test (ttest_1samp)     ", p_value, check_significance(significance, p_value))

# one sample wilcoxon-test
z_statistic, p_value = wilcoxon(daily_intake - 7725)
print( "one-sample wilcoxon-test (wilcoxon) ", p_value, check_significance(significance, p_value))

energ = np.array([
# energy expenditure in mJ and stature (0=obese, 1=lean)
[9.21, 0],
[7.53, 1],
[7.48, 1],
[8.08, 1],
[8.09, 1],
[10.15, 1],
[8.40, 1],
[10.88, 1],
[6.13, 1],
[7.90, 1],
[11.51, 0],
[12.79, 0],
[7.05, 1],
[11.85, 0],
[9.97, 0],
[7.48, 1],
[8.79, 0],
[9.69, 0],
[9.68, 0],
[7.58, 1],
[9.19, 0],
[8.11, 1]])

# similar to expend ~ stature in R
group1 = energ[:, 1] == 0
group1 = energ[group1][:, 0]
group2 = energ[:, 1] == 1
group2 = energ[group2][:, 0]

# two-sample t-test
# null hypothesis: the two groups have the same mean
# this test assumes the two groups have the same variance...
# (can be checked with tests for equal variance)
# independent groups: e.g., how boys and girls fare at an exam
# dependent groups: e.g., how the same class fare at 2 different exams
t_statistic, p_value = ttest_ind(group1, group2)

# p_value < 0.05 => alternative hypothesis:
# they don't have the same mean at the 5% significance level
print( "two-sample t-test (ttest_ind)", p_value, check_significance(significance, p_value))

# two-sample wilcoxon test
# a.k.a Mann Whitney U
u, p_value = mannwhitneyu(group1, group2)
print( "two-sample wilcoxon-test (mannwhitneyu)", p_value, check_significance(significance, p_value))

# pre and post-menstrual energy intake
intake = np.array([
[5260, 3910],
[5470, 4220],
[5640, 3885],
[6180, 5160],
[6390, 5645],
[6515, 4680],
[6805, 5265],
[7515, 5975],
[7515, 6790],
[8230, 6900],
[8770, 7335],
])

pre = intake[:, 0]
post = intake[:, 1]

# paired t-test: doing two measurments on the same experimental unit
# e.g., before and after a treatment
t_statistic, p_value = ttest_1samp(post - pre, 0)

# p < 0.05 => alternative hypothesis:
# the difference in mean is not equal to 0
print("paired t-test (ttest_1samp)", p_value, check_significance(significance, p_value))

# alternative to paired t-test when data has an ordinary scale or when not
# normally distributed
z_statistic, p_value = wilcoxon(post - pre)

print("paired wilcoxon-test (wilcoxon)", p_value, check_significance(significance, p_value))

one-sample t-test (ttest_1samp)      0.0181372351761 False
one-sample wilcoxon-test (wilcoxon)  0.0261571823293 False
two-sample t-test (ttest_ind) 0.00079899821117 False
two-sample wilcoxon-test (mannwhitneyu) 0.00106080669294 False
paired t-test (ttest_1samp) 3.05902094293e-07 False
paired wilcoxon-test (wilcoxon) 0.00333001391175 False
